# Advent of Code 2024

## Contents
<table>
<tr><td>

- [Day 1](#day-1)
- [Day 2](#day-2)
- [Day 3](#day-3)
- [Day 4](#day-4)
- [Day 5](#day-5)

</td><td>

- [Day 6](#day-6)
- [Day 7](#day-7)
- [Day 8](#day-8)
- [Day 9](#day-9)
- [Day 10](#day-10)

</td><td>

- [Day 11](#day-11)
- [Day 12](#day-12)
- [Day 13](#day-13)
- [Day 14](#day-14)
- [Day 15](#day-15)

</td><td>

- [Day 16](#day-16)
- [Day 17](#day-17)
- [Day 18](#day-18)
- [Day 19](#day-19)
- [Day 20](#day-20)

</td><td>

- [Day 21](#day-21)
- [Day 22](#day-22)
- [Day 23](#day-23)
- [Day 24](#day-24)
- [Day 25](#day-25)

</td></tr>
</table>

## Boilerplate

In [24]:
# SETUP #
# Currently just what was needed in 2023, will adjust as we go
import sys
import math
import operator
import copy
import numpy as np
import cProfile
from itertools import compress, combinations
from functools import reduce, cmp_to_key, cache
from dataclasses import dataclass
from typing import Tuple
from collections import Counter

@dataclass
class DayData:
    input: str
    test_input: str
    test_solutions: Tuple[int, int]

# Load input and solution data #
def init_day(day, test_solutions) -> DayData:

    def read_file(path):
        with open(path, mode="rt") as f:
            return f.read()

    return DayData(
        input = read_file(f"inputs/{day}.txt"),
        test_input = read_file(f"test_inputs/{day}.txt"),
        test_solutions = test_solutions
    )

# Run and test code #
def test(test: int, solution: int):
    return "Success!" if test == solution else f"Failed. Expected {solution}, but got {test}."

def run_test(func_to_run, day_data):
    print(f"Test Part 1: {test(func_to_run(1,day_data.test_input), day_data.test_solutions[0])}")
    print(f"Test Part 2: {test(func_to_run(2,day_data.test_input), day_data.test_solutions[1])}")

def run_real(func_to_run, day_data):
    %time print(f"Answer Part 1: {func_to_run(1,day_data.input)}")
    %time print(f"Answer Part 2: {func_to_run(2,day_data.input)}")


## Day 1

[Link to puzzle](https://adventofcode.com/2024/day/1)

In [28]:
# DAY 1 #
def run(part,i):
    left, right = zip(*(map(int, line.split()) for line in i.split('\n'))) # Divide inputs into left and right columns

    def part_1():
        differences = [abs(l - r) for l, r in zip(sorted(left), sorted(right))]
        return sum(differences)

    def part_2():
        # Similarity - multiply each number in left list by the number of times it appears in the right list
        right_counts = Counter(right) # originally used right.count(), but this is significantly faster (13ms -> 1ms)
        return sum(value * right_counts[value] for value in left) 

    return part_1() if part == 1 else part_2()

day_data = init_day(day=1, test_solutions=(11, 31))
run_test(run, day_data)
run_real(run, day_data)

Test Part 1: Success!
Test Part 2: Success!
Answer Part 1: 2815556
CPU times: user 1.76 ms, sys: 20 µs, total: 1.78 ms
Wall time: 1.78 ms
Answer Part 2: 23927637
CPU times: user 1.02 ms, sys: 0 ns, total: 1.02 ms
Wall time: 1.03 ms


## Day 2

[Link to puzzle](https://adventofcode.com/2024/day/2)

In [ ]:
# DAY 2 #
init_day(day=2, test_solutions=(0, 0))

def run(part,i):
    return 0

%time run_test(run)
%time run_real(run)

## Day 3

[Link to puzzle](https://adventofcode.com/2024/day/3)

In [ ]:
# DAY 3 #
init_day(day=3, test_solutions=(0, 0))

def run(part,i):
    return 0

%time run_test(run)
%time run_real(run)

## Day 4

[Link to puzzle](https://adventofcode.com/2024/day/4)

In [ ]:
# DAY 4 #
init_day(day=4, test_solutions=(0, 0))

def run(part,i):
    return 0

%time run_test(run)
%time run_real(run)

## Day 5

[Link to puzzle](https://adventofcode.com/2024/day/5)

In [ ]:
# DAY 5 #
init_day(day=5, test_solutions=(0, 0))

def run(part,i):
    return 0

%time run_test(run)
%time run_real(run)

## Day 6

[Link to puzzle](https://adventofcode.com/2024/day/6)

In [ ]:
# DAY 6 #
init_day(day=6, test_solutions=(0, 0))

def run(part,i):
    return 0

%time run_test(run)
%time run_real(run)

## Day 7

[Link to puzzle](https://adventofcode.com/2024/day/7)

In [ ]:
# DAY 7 #
init_day(day=7, test_solutions=(0, 0))

def run(part,i):
    return 0

%time run_test(run)
%time run_real(run)